In [ ]:
class CustomDataset(Dataset):
    def __init__(self, path, subset='train', transform=None):
        
        self.path = path
        self.folder = folder
        self.transform = transform
        
        self.image_paths = []
        self.labels = []

        folder_path = os.path.join(path, subset)
        
        if os.path.isdir(folder_path):
            for class_name in os.listdir(folder_path):
                dir_path = os.path.join(folder_path, class_name)
                
                if os.path.isdir(dir_path):
                    label = class_name  # 폴더명을 레이블로 사용
                    for filename in os.listdir(dir_path):
                        if filename.endswith('.png'):
                            self.image_paths.append(os.path.join(dir_path, filename))
                            self.labels.append(label)
        else:
            raise ValueError(f'Invalid subset: {subset}')

            
    def __len__(self):
        return len(self.image_paths)

    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        
        image = Image.open(image_path)

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
class Dataloader():
    def __init__(self, path):
        normalize = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        
        transform_train = transforms.Compose([
                        transforms.Resize((512, 512)),
                        transforms.ToTensor(),
                        normalize
                        ])
        transform_test = transforms.Compose([
                        transforms.Resize((512, 512)),
                        transforms.ToTensor(),
                        normalize
                        ])
        
        trainset = CustomDataset(path, subset='train', transform=transform_train)
        valset = CustomDataset(path, subset='val', transform=transform_train)
        testset = CustomDataset(path, subset='test', transform=transform_test)
        
        self.train_loader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=4)
        self.val_loader = DataLoader(valset, batch_size=64, shuffle=True, num_workers=4)
        self.test_loader = DataLoader(testset, batch_size=64, shuffle=True, num_workers=4)
        
    def getloader(self):
        return self.train_loader, self.val_loader, self.test_loader


In [ ]:
# 모델 훈련
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    
    for images, labels in train_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        
        # 순전파
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # 역전파 및 최적화
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # 정확도 계산
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)
        
        # 통계 출력
        running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(train_dataset)
    epoch_accuracy = correct_predictions / total_predictions * 100
    
    print(f'Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_accuracy:.2f}%')

SyntaxError: invalid syntax (2232192826.py, line 1)

In [ ]:
import torch
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지 크기 조정
    transforms.ToTensor()  # 이미지를 텐서로 변환
])

In [ ]:
from torchvision.datasets import ImageFolder

train_dataset = ImageFolder('데이터셋 폴더/train', transform=transform)
val_dataset = ImageFolder('데이터셋 폴더/val', transform=transform)
test_dataset = ImageFolder('데이터셋 폴더/test', transform=transform)


In [ ]:
from torch.utils.data import DataLoader

batch_size = 32

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)


In [ ]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc = nn.Linear(32 * 56 * 56, 2)  # 예측할 클래스 수에 맞게 설정

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [ ]:
model = CNN()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 모델 학습
for epoch in range(num_epochs):
    for images, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # 검증 데이터셋을 사용하여 성능 평가
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in val_dataloader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        accuracy = correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Accuracy: {accuracy:.4f}')


In [ ]:
# 테스트 데이터셋을 사용하여 모델 평가
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_dataloader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    accuracy = correct / total
    print(f'Test Accuracy: {accuracy:.4f}')
